In [1]:
import os
import json
import time
from dotenv import load_dotenv
from google import genai
from google.genai import types

# --- 1. SETUP ---
# Adjust path to find .env (Assuming notebook is in Finetuning/notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
env_path = os.path.join(project_root, ".env")
load_dotenv(env_path)

api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    print(f"❌ Error: GOOGLE_API_KEY not found. Checked: {env_path}")
else:
    print("✅ API Key found.")

# Configuration
MODEL_ID = "gemini-3-flash-preview"
PROMPT_PATH = "../prompts/extraction.txt"

# --- 2. HELPERS ---
def clean_json_response(text):
    text = text.strip()
    if text.startswith("```json"): text = text[7:]
    elif text.startswith("```"): text = text[3:]
    if text.endswith("```"): text = text[:-3]
    return text.strip()

# --- 3. DUMMY DATA ---
test_nl = """
When preparing a workstation tailored for data-intensive simulations, you’re expected to
select a processor, a graphics card, and a storage unit from a shortlist provided by the
procurement office. Each category contains exactly three items, all of which vary in
cost and advertised performance ratings. While only one item from each class can be
chosen, it's not as simple as picking the best in each—certain budgetary and performance
considerations are in play.
Let’s start with the CPUs: the X9 offers moderate speed (score 95) for $300, the ZetaCore is the
highest-performing chip (score 120) and costs $500, and the Alpha8, while slightly below
ZetaCore in speed (score 110), still carries a premium at $450. Meanwhile, your GPU choices
include QuasarRTX (130 points, $700), NovaX (125 points, $600), and the more modest GammaGTX
(90 points, $450). For storage, there’s UltraSSD (100 points, $250), FusionDrive (95 points,
$300), and the budget-tier MegaHDD (60 points, $200).
One of the directives from IT leadership is that your most costly component must also be
the one delivering the greatest performance in your final selection—presumably to justify
budget overruns. However, it's frowned upon to pick the single highest-rated part from each
of the three categories; evidently, they want to avoid over-spec’d builds. There’s also a
guideline about balance: while you don’t need to match prices perfectly, no component
should differ in cost from another by more than $250. Consider that a soft constraint, they'll
reject your build if that balance is off.
Also, while the graphics team claims that a powerful GPU helps with rendering, you’re not
allowed to have a GPU that outperforms the CPU. A constraint buried in the procurement
notes says that the Alpha8 CPU isn’t validated to work with the NovaX GPU due to BIOS
issues. And for some reason, the FusionDrive isn’t available unless you’re pairing it
with the ZetaCore.
Despite the abundance of high-performance hardware, you’ll need to select carefully to
stay within these constraints.
"""
test_asp = """
% Components: CPUs = {X9, ZetaCore, Alpha8}, GPUs = {QuasarRTX, NovaX, GammaGTX},Storage = {UltraSSD, MegaHDD, FusionDrive}.
cpu("X9"; "ZetaCore"; "Alpha8").
gpu("QuasarRTX"; "NovaX"; "GammaGTX").
storage("UltraSSD"; "MegaHDD"; "FusionDrive").

% Select exactly one CPU, one GPU, and one storage device.
1 { selected_cpu(C) : cpu(C) } 1.
1 { selected_gpu(G) : gpu(G) } 1.
1 { selected_storage(S) : storage(S) } 1.

% CPU Costs: [X9: 300, ZetaCore: 500, Alpha8: 450].
cost("X9", 300; "ZetaCore", 500; "Alpha8", 450).

% GPU Costs: [QuasarRTX: 700, NovaX: 600, GammaGTX: 450].
cost("QuasarRTX", 700; "NovaX", 600; "GammaGTX", 450).

% Storage Costs: [UltraSSD: 250, MegaHDD: 200, FusionDrive: 300].
cost("UltraSSD", 250; "MegaHDD", 200; "FusionDrive", 300).

% CPU Performance: [X9: 95, ZetaCore: 120, Alpha8: 110].
perf("X9", 95; "ZetaCore", 120; "Alpha8", 110).

% GPU Performance: [QuasarRTX: 130, NovaX: 125, GammaGTX: 90].
perf("QuasarRTX", 130; "NovaX", 125; "GammaGTX", 90).

% Storage Performance: [UltraSSD: 100, MegaHDD: 60, FusionDrive: 95].
perf("UltraSSD", 100; "MegaHDD", 60; "FusionDrive", 95).

% The most expensive selected component must also be the one with the highest performance among selected parts.
selected(X) :- selected_cpu(X).
selected(X) :- selected_gpu(X).
selected(X) :- selected_storage(X).

max_cost(C) :- cost(C, V), V = #max {V1 : cost(C1, V1), selected(C1)}.
max_perf(C) :- perf(C, V), V = #max {V1 : perf(C1, V1), selected(C1)}.
:- selected(C), max_cost(C), not max_perf(C).

% You may not select all three top-performing components.
top_cpu(C) :- cpu(C), perf(C, V), V = #max {V1 : perf(C1, V1), cpu(C1)}.
top_gpu(C) :- gpu(C), perf(C, V), V = #max {V1 : perf(C1, V1), gpu(C1)}.
top_storage(C) :- storage(C), perf(C, V), V = #max {V1 : perf(C1, V1), storage(C1)}.

:- selected_cpu(C), top_cpu(C), selected_gpu(G), top_gpu(G), selected_storage(S), top_storage(S).

% The difference in cost between any two selected components must be ≤ 250.
:- selected(C1), selected(C2), cost(C1, V1), cost(C2, V2), V1 - 250 >= V2.

% The selected GPU must not have higher performance than the selected CPU.
:- selected_cpu(C), selected_gpu(G), perf(C, PC), perf(G, PG), PG > PC.

% Alpha8 is incompatible with NovaX.
:- selected_cpu("Alpha8"), selected_gpu("NovaX").

% FusionDrive can only be used with ZetaCore.
:- selected_storage("FusionDrive"), not selected_cpu("ZetaCore").

"""
test_tone = "Didactic/Instructional"

# --- 4. EXECUTION ---
if not os.path.exists(PROMPT_PATH):
    print(f"❌ Critical: Prompt file not found at {PROMPT_PATH}")
else:
    try:
        # A. READ
        with open(PROMPT_PATH, "r", encoding="utf-8") as f:
            raw_template = f.read()
        print(f"Loaded prompt template ({len(raw_template)} chars).")

        # B. FORMAT (Catches brace errors)
        formatted_prompt = raw_template.format(
            nl_content=test_nl,
            asp_content=test_asp,
            selected_tone=test_tone
        )
        print("✅ Python String Formatting Successful (No loose braces found).")

        # C. API CALL
        print(f"🚀 Sending request to {MODEL_ID}...")
        client = genai.Client(api_key=api_key)
        
        start_time = time.time()
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=[formatted_prompt], # Must be a list
            config=types.GenerateContentConfig(
                response_mime_type="application/json",
                temperature=1
            )
        )
        elapsed = time.time() - start_time
        print(f"✅ API Response received in {elapsed:.2f}s.")

        # D. JSON PARSE
        cleaned_text = clean_json_response(response.text)
        data = json.loads(cleaned_text)
        
        print("\n✅ JSON Parsing Successful. Result:")
        print("-" * 40)
        print(json.dumps(data, indent=2))
        print("-" * 40)

        # E. VALIDATION
        required_keys = ["instance_template", "generator", "hard_constraints", "soft_constraints", "global_objective"]
        missing = [k for k in required_keys if k not in data]
        
        if missing:
            print(f"⚠️ Warning: The JSON is valid, but missing specific keys: {missing}")
        else:
            print("🎉 Full Success: Structure matches requirements perfectly.")

    except KeyError as e:
        print(f"\n❌ FORMATTING ERROR: Missing Variable or Unescaped Brace")
        print(f"Python found a single brace {{...}} containing: '{e.args[0]}'")
        print("Fix: Go to extraction.txt and double-escape literal braces like #count {{ ... }}")
        
    except Exception as e:
        print(f"\n❌ RUNTIME ERROR: {e}")

✅ API Key found.
Loaded prompt template (4059 chars).
✅ Python String Formatting Successful (No loose braces found).
🚀 Sending request to gemini-3-flash-preview...
✅ API Response received in 29.38s.

✅ JSON Parsing Successful. Result:
----------------------------------------
{
  "instance_template": "cpu(\"X9\"). cpu(\"ZetaCore\"). cpu(\"Alpha8\"). gpu(\"QuasarRTX\"). gpu(\"NovaX\"). gpu(\"GammaGTX\"). storage(\"UltraSSD\"). storage(\"MegaHDD\"). storage(\"FusionDrive\"). cost(\"X9\", 300). cost(\"ZetaCore\", 500). cost(\"Alpha8\", 450). cost(\"QuasarRTX\", 700). cost(\"NovaX\", 600). cost(\"GammaGTX\", 450). cost(\"UltraSSD\", 250). cost(\"MegaHDD\", 200). cost(\"FusionDrive\", 300). perf(\"X9\", 95). perf(\"ZetaCore\", 120). perf(\"Alpha8\", 110). perf(\"QuasarRTX\", 130). perf(\"NovaX\", 125). perf(\"GammaGTX\", 90). perf(\"UltraSSD\", 100). perf(\"MegaHDD\", 60). perf(\"FusionDrive\", 95).",
  "generator": "1 { selected_cpu(C) : cpu(C) } 1. 1 { selected_gpu(G) : gpu(G) } 1. 1 { sel

In [1]:
import sys
import os
import random
import json

# 1. Add the parent directory (Finetuning) to sys.path to import utils_ft
# This assumes the notebook is running in Finetuning/notebooks/
sys.path.append('..')

from utils_ft import (
    get_benchmark_data_path, 
    get_processed_data_path, 
    get_scripts_path,
    to_relative
)

# 2. Add scripts folder to path so we can import the converter
sys.path.append(get_scripts_path())

from data_converter import main as run_converter

# Define Paths using the Utils
data_in = get_benchmark_data_path()
data_out = get_processed_data_path()

print(f"Input:  {to_relative(data_in)}")
print(f"Output: {to_relative(data_out)}")

GOOGLE_API_KEY present? True
Input:  Finetuning/data/Benchmark Data
Output: Finetuning/data


In [2]:
run_converter(data_in, data_out)

Scanning Finetuning/data/Benchmark Data...

--- Processing Training Set (43 Groups) ---
> room_booking_easy_1
  [Extraction] Querying Gemini for structure...
You are an expert in Answer Set Programming (specifically Clingo 5+ syntax). 
I will give you a full problem description (NL) and a full ASP solution.

Your task is to DECONSTRUCT this solution into the specific structural components required for a scheduler system.
Return the result as a valid JSON object.

### CODE REFACTORING & OPTIMIZATION (CRITICAL)
The input ASP code might be old, inefficient, or syntactically loose. Do NOT just copy-paste it.
**You have full authority to rewrite the logic** to ensure the highest quality training data:
1.  **Modernize:** Use modern Clingo 5+ syntax (e.g., `#count`, `#sum`). Replace deprecated constructs.
2.  **Optimize:** If a constraint is written inefficiently (e.g., using unnecessary helper predicates for simple checks), refactor it to be concise and readable.
3.  **Correctness:** If the 

: 

In [ ]:
train_file = os.path.join(data_out, "train.jsonl")

# 1. Load all lines into memory (efficient enough for small-medium datasets)
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Pick a random line
if lines:
    random_line = random.choice(lines)
    sample = json.loads(random_line)

    print(f"--- RANDOM SAMPLE (Total Examples: {len(lines)}) ---")
    
    # 3. Iterate through messages and print nicely
    for msg in sample['messages']:
        role = msg['role'].upper()
        content = msg['content']
        
        print(f"\n[{role}]:")
        print("-" * 20)
        print(content)
        print("=" * 60)
else:
    print("Dataset file is empty.")

--- RANDOM SAMPLE (Total Examples: 387) ---

[SYSTEM]:
--------------------
You are an expert in Answer Set Programming. Task: Convert natural language instance descriptions into ASP facts and domain definitions. Output: Provide ONLY the valid Clingo code. Do not provide any explanation, comments, or markdown formatting.

[USER]:
--------------------
Problem Description:
A drone is deployed from headquarters at point 1 to scan a couple of points on a short
path network. There are four locations in total. Connections exist such that 1 links to 2
and 3; 2 links to 4. The drone is only allowed to fly between adjacent locations.
It has exactly 5 time units to complete its path, with each move
taking 1 unit of time. The scanning task requires it to visit locations 3 and 4, in any 
order. It begins at 1.

### TASK
Create the instance template (define predicates and domains).

[ASSISTANT]:
--------------------
loc(1..4). edge(1, 2). edge(1, 3). edge(2, 4). time(0..5).
